# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, r2_score

In [3]:
from explainerdashboard.explainers import *
from explainerdashboard.dashboards import *
from explainerdashboard.datasets import *

# RandomForestRegressor

In [4]:
X_train, y_train, X_test, y_test = titanic_fare()
train_names, test_names = titanic_names()

model = RandomForestRegressor(n_estimators=50, max_depth=5)
model.fit(X_train, y_train)

explainer = RandomForestRegressionBunch(model, X_test, y_test, r2_score, 
                               cats=['Sex', 'Deck', 'Embarked'],
                               idxs=test_names, units="$")

In [ ]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=False,
                        shap_dependence=True,
                        shap_interaction=True,
                        shadow_trees=False)
db.run(8051)

Calculating predictions...
Calculating prediction probabilities...
Calculating importances...
Running Model Explainer on http://localhost:8051
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Jan/2020 20:12:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:12:05] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:12:05] "GET /assets/bootstrap.css?m=1570631370.0 HTTP/1.1" 304 -
127.0.0.1 - - [25/Jan/2020 20:12:05] "GET /assets/stylesheet.css?m=1570631370.0 HTTP/1.1" 304 -
127.0.0.1 - - [25/Jan/2020 20:12:05] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:12:05] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:12:05] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.1.1&m=1566138668 HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:12:05] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_componen

# RandomForestClassifier:

In [23]:
X_train, y_train, X_test, y_test = titanic_survive()
train_names, test_names = titanic_names()

model = RandomForestClassifier(n_estimators=50, max_depth=5)
model.fit(X_train, y_train)


explainer = RandomForestClassifierBunch(model, X_test, y_test, roc_auc_score, 
                               cats=['Sex', 'Deck', 'Embarked'],
                               idxs=test_names, #names of passengers 
                               labels=['Not survived', 'Survived'])

In [ ]:
db = ExplainerDashboard(explainer,
                        model_summary=False,
                        contributions=False,
                        shap_dependence=False,
                        shap_interaction=True,
                        shadow_trees=False)
db.run(8052)

Running Model Explainer on http://localhost:8052
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Jan/2020 20:06:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:06:27] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:06:27] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:06:27] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1566138683 HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:06:27] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.1.1&m=1566138668 HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:06:27] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.min.js?v=0.7.0&m=1566138674 HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:06:27] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1566138672 HTTP/1.1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2890, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Sex_female'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception

127.0.0.1 - - [25/Jan/2020 20:06:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:06:33] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2890, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Sex_female'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception

127.0.0.1 - - [25/Jan/2020 20:06:33] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [25/Jan/2020 20:06:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:07:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:07:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:07:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:07:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:07:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:07:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:07:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:07:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:07:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 20:07:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.

# XGBoostClassifier

In [ ]:
X_train, y_train, X_test, y_test = titanic_survive()
train_names, test_names = titanic_names()

xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

explainer = TreeModelClassifierBunch(xgb_model, X_test, y_test, roc_auc_score, 
                                   cats=['Sex', 'Cabin', 'Embarked'],
                                   idxs=test_names, #names of passengers 
                                   labels=['Not survived', 'Survived'])

db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False)
db.run(8052)